### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Ocular_Melanomas/GSE15309'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Set `is_gene_available` to `True`
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability for 'Ocular_Melanomas'
# Based on available entries, there isn't a specific mention of 'Ocular_Melanomas'
trait_row = None

# 2.1 Data Availability for 'age'
# Age data is likely under index 2 with keys like 'age: 34', 'age: 35 yrs', etc.
if any(key.startswith('age:') for key in sample_characteristics[2]):
    age_row = 2
    
# 2.1 Data Availability for 'gender'
# Gender data is likely under index 3 with keys like 'sex: male', 'sex: female', etc.
if any(key.startswith('sex:') for key in sample_characteristics[3]):
    gender_row = 3

# 2.3 Data Type Conversion
# Converting 'Ocular_Melanomas'
def convert_trait(value):
    return None  # No specific conversion as trait_row is None

# Converting 'age'
def convert_age(value):
    try:
        age_value = value.split(':')[-1].strip()
        return float(age_value.split()[0].replace('yrs', '').replace('years', '').replace('months', '')) if age_value else None
    except:
        return None
    
# Converting 'gender'
def convert_gender(value):
    gender_value = value.split(':')[-1].strip()
    if gender_value.lower() == 'male':
        return 1
    elif gender_value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
def save_cohort_info(study_id, file_path, is_gene_available, clinical_data_available):
    metadata = {
        "study_id": study_id,
        "file_path": file_path,
        "is_gene_available": is_gene_available,
        "clinical_data_available": clinical_data_available
    }
    with open(file_path, 'w') as file:
        json.dump(metadata, file)
        
save_cohort_info('GSE15309', './preprocessed/Ocular_Melanomas/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Ocular_Melanomas', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Ocular_Melanomas/trait_data/GSE15309.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
